In [42]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import json
import altair as alt
from altair_saver import save

In [43]:
WD = "/Users/alex/Documents/RABIES"
PATH = os.path.join(WD, "RESULTS")
#PATH = WD
JSON_FILES = [os.path.join(PATH, file) for file in os.listdir(PATH) 
              if file.endswith(".BUSTEDS.json") and os.path.isfile(os.path.join(PATH, file))]
pvalueThreshold = 0.05
print("# Number of BUSTED[S] json files", len(JSON_FILES))

# Number of BUSTED[S] json files 25


In [44]:
def get_JSONData(json_file):
    if os.stat(json_file).st_size == 0:
         print('# File is empty:', json_file)
    else:
        #print('File is not empty')
        with open(json_file, "r") as in_d:
             json_data = json.load(in_d)
        #end with
        in_d.close()
        return json_data
    #end if
#end method

def num_selected_sites(json_data, pvalueThreshold, Positive=True):
    columns = json_data["MLE"]["headers"]
    headers = [x[0] for x in columns]
    
    data = json_data["MLE"]["content"]["0"]
    df = pd.DataFrame(data, columns=headers, dtype = float)
    df["omega"] = df["beta"] / df["alpha"]
    df_results = df[df["p-value"] <= pvalueThreshold]
    
    # boolean
    if Positive == True:
        positive_sites = df_results[df_results["omega"] > 1.0]
        return len(positive_sites)
    elif Positive == False:
        negative_sites = df_results[df_results["omega"] < 1.0]
        return len(negative_sites)
    else:
        return 0
    #end if
#end method


In [45]:
df_dict = {}
count = 0

for file in JSON_FILES:
    json_data = get_JSONData(file)
    basename = os.path.basename(file)
    basename = basenameH.replace(".fas.BUSTEDS.json", "")
    basename = basename.split("_")
    Species = basename[2]
    Virus = " ".join([basename[3], basename[4]])
    Gene = basename[5]
    
    df_dict[count] = {"Virus": Virus}
    # Number of sites
    sites = json_data["input"]["number of sites"]
    # Number of sequences
    seqs = json_data["input"]["number of sequences"] 
    df_dict[count].update({"Species": Species,
                           "Gene": Gene, 
                           "N": seqs,
                           "Codons": sites})
    # Omegas and proportions
    data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
    w1 = round(data["0"]["omega"], 3)
    p1 = round(data["0"]["proportion"], 3)
    w2 = round(data["1"]["omega"], 3)
    p2 = round(data["1"]["proportion"], 3)
    w3 = round(data["2"]["omega"], 3)
    p3 = round(data["2"]["proportion"], 3)
    df_dict[count].update({"w1": w1,
                          "p1": p1})
    df_dict[count].update({"w2": w2,
                          "p2": p2})
    df_dict[count].update({"w3": w3,
                          "p3": p3})
    # Test results
    LRT = round(json_data["test results"]["LRT"], 1)
    pval = round(json_data["test results"]["p-value"], 3)
    df_dict[count].update({"LRT": LRT,
                          "pvalue": pval})
    count += 1
#end for

In [37]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df.index += 1
df

,Virus,Species,Gene,N,Codons,w1,p1,w2,p2,w3,p3,LRT,pvalue
1,Rables Lyssavirus,Cat,L,17,2129,0.023,0.995,0.842,0.005,2.324000e+00,0.000,0.0,0.500
2,Rabies Lyssavirus,Human,M,28,203,0.039,0.998,0.039,0.001,1.120100e+01,0.001,1.8,0.204
3,Rables Lyssavirus,Raccoon,P,305,302,0.000,0.729,0.485,0.269,6.810000e+00,0.002,0.3,0.439
4,Rables Lyssavirus,Cat,G,90,566,0.000,0.448,0.165,0.549,6.484000e+00,0.002,-0.7,0.500
5,Rables Lyssavirus,Dog,N,2667,1024,0.005,0.656,0.437,0.334,1.396200e+01,0.009,434.0,0.000
6,Rables Lyssavirus,Bat,P,264,310,0.004,0.299,0.098,0.581,1.000000e+00,0.120,0.0,0.500
7,Rabies Lyssavirus,Human,L,30,2129,0.002,0.811,0.127,0.189,9.853290e+02,0.000,21.6,0.000
8,Rables Lyssavirus,Human,G,122,528,0.001,0.535,0.155,0.464,1.883800e+01,0.000,1.9,0.191
9,Rables Lyssavirus,Cat,M,16,203,0.030,0.501,0.033,0.497,3.030000e+00,0.002,0.2,0.446
10,Rables Lyssavirus,Raccoon,N,384,513,0.001,0.581,0.069,0.415,1.762300e+01,0.004,86.6,0.000


In [38]:
df = df.sort_values(by=['Species'])
df = df.reset_index()
#df.drop('index', axis=1, inplace=True)
df.index += 1
df.drop('index', axis=1, inplace=True)
df

,Virus,Species,Gene,N,Codons,w1,p1,w2,p2,w3,p3,LRT,pvalue
1,Rables Lyssavirus,Bat,L,125,2129,0.009,0.923,0.463,0.077,4.675522e+03,0.000,48.6,0.000
2,Rables Lyssavirus,Bat,M,77,203,0.017,0.897,0.336,0.103,1.002000e+00,0.000,0.0,0.500
3,Rables Lyssavirus,Bat,G,414,575,0.000,0.310,0.099,0.614,1.000000e+00,0.076,0.0,0.500
4,Rables Lyssavirus,Bat,P,264,310,0.004,0.299,0.098,0.581,1.000000e+00,0.120,0.0,0.500
5,Rables Lyssavirus,Bat,N,983,622,0.000,0.439,0.100,0.557,1.659400e+01,0.004,147.1,0.000
6,Rables Lyssavirus,Cat,P,28,341,0.000,0.326,0.269,0.671,2.459800e+01,0.003,12.0,0.001
7,Rables Lyssavirus,Cat,N,115,489,0.001,0.498,0.094,0.502,9.999999e+09,0.000,31.9,0.000
8,Rables Lyssavirus,Cat,M,16,203,0.030,0.501,0.033,0.497,3.030000e+00,0.002,0.2,0.446
9,Rables Lyssavirus,Cat,L,17,2129,0.023,0.995,0.842,0.005,2.324000e+00,0.000,0.0,0.500
10,Rables Lyssavirus,Cat,G,90,566,0.000,0.448,0.165,0.549,6.484000e+00,0.002,-0.7,0.500


In [32]:
print(df.to_markdown())

|    | Virus             | Species   | Gene   |    N |   Codons |    w1 |    p1 |    w2 |    p2 |       w3 |    p3 |   LRT |   pvalue |
|---:|:------------------|:----------|:-------|-----:|---------:|------:|------:|------:|------:|---------:|------:|------:|---------:|
|  1 | Rables Lyssavirus | Bat       | L      |  125 |     2129 | 0.009 | 0.923 | 0.463 | 0.077 | 4675.52  | 0     |  48.6 |    0     |
|  2 | Rables Lyssavirus | Bat       | M      |   77 |      203 | 0.017 | 0.897 | 0.336 | 0.103 |    1.002 | 0     |   0   |    0.5   |
|  3 | Rables Lyssavirus | Bat       | G      |  414 |      575 | 0     | 0.31  | 0.099 | 0.614 |    1     | 0.076 |   0   |    0.5   |
|  4 | Rables Lyssavirus | Bat       | P      |  264 |      310 | 0.004 | 0.299 | 0.098 | 0.581 |    1     | 0.12  |   0   |    0.5   |
|  5 | Rables Lyssavirus | Bat       | N      |  983 |      622 | 0     | 0.439 | 0.1   | 0.557 |   16.594 | 0.004 | 147.1 |    0     |
|  6 | Rables Lyssavirus | Cat       | P      | 

## Save dataframe

In [41]:
df.to_csv(os.path.join(WD, "TABLES", "RABIES_BUSTEDS.csv"))